In [1]:
import pandas as pd


def read_label_map(label_map_path):

    item_id = None
    item_name = None
    items = {}

    with open(label_map_path, "r") as file:
        for line in file:
            line.replace(" ", "")
            if line == "item{":
                pass
            elif line == "}":
                pass
            elif "label_id" in line:
                item_id = int(line.split(":", 1)[1].strip()) - 1
            elif "name" in line:
                item_name = line.split(":", 1)[1].replace("'", "").replace('"', "").strip()

            if item_id is not None and item_name is not None:
                items[item_name] = item_id
                item_id = None
                item_name = None

    return items

In [2]:
activities = read_label_map(r"D:\Projects\ava\ava_v2.1\ava_action_list_v2.1.pbtxt")

In [9]:
print({f'{k}': v.split(" ")[0] for v, k in activities.items()})

{'0': 'bend/bow', '1': 'crawl', '2': 'crouch/kneel', '3': 'dance', '4': 'fall', '5': 'get', '6': 'jump/leap', '7': 'lie/sleep', '8': 'martial', '9': 'run/jog', '10': 'sit', '11': 'stand', '12': 'swim', '13': 'walk', '14': 'answer', '15': 'brush', '16': 'carry/hold', '17': 'catch', '18': 'chop', '19': 'climb', '20': 'clink', '21': 'close', '22': 'cook', '23': 'cut', '24': 'dig', '25': 'dress/put', '26': 'drink', '27': 'drive', '28': 'eat', '29': 'enter', '30': 'exit', '31': 'extract', '32': 'fishing', '33': 'hit', '34': 'kick', '35': 'lift/pick', '36': 'listen', '37': 'open', '38': 'paint', '39': 'play', '40': 'play', '41': 'play', '42': 'point', '43': 'press', '44': 'pull', '45': 'push', '46': 'put', '47': 'read', '48': 'ride', '49': 'row', '50': 'sail', '51': 'shoot', '52': 'shovel', '53': 'smoke', '54': 'stir', '55': 'take', '56': 'text', '57': 'throw', '58': 'touch', '59': 'turn', '60': 'watch', '61': 'work', '62': 'write', '63': 'fight/hit', '64': 'give/serve', '65': 'grab', '66': 

In [23]:
target_df = pd.read_csv(r"D:\Projects\ava\ava_v2.1\ava_val_v2.1.csv", names=['id', 'timestamp', 'x1', 'y1', 'x2', 'y2', 'action_id', 'person_id'])
stochastic_df = pd.read_csv(r"D:\Projects\ava\ava_baseline_detections_val_v2.1\ava_baseline_detections_val_v2.1.csv", names=['id', 'timestamp', 'x1', 'y1', 'x2', 'y2', 'action_id', 'score'])

In [24]:
import numpy as np

def reconstruct_prob_vector(k, p_k, num_classes, alpha=0.3):
    """
    k           : index of predicted class (0-based)
    p_k         : known probability of the predicted class
    num_classes : total number of classes
    alpha       : Dirichlet concentration parameter
    """
    if not (0 < p_k < 1):
        p_k = abs(p_k - 5e-5)

    rest = num_classes - 1
    q = np.random.dirichlet([alpha] * rest)
    p = np.zeros(num_classes)
    p[k] = p_k
    p_rest = (1 - p_k) * q
    p[np.arange(num_classes) != k] = p_rest

    return p

In [31]:
len(stochastic_df[stochastic_df["id"] == "1j20qq1JyX4"]) / len(target_df[target_df["id"] == "1j20qq1JyX4"])

10.743897026187305

In [35]:
target_df[target_df["id"] == "1j20qq1JyX4"]

,id,timestamp,x1,y1,x2,y2,action_id,person_id
0,1j20qq1JyX4,902,0.002,0.118,0.714,0.977,12,1
1,1j20qq1JyX4,902,0.002,0.118,0.714,0.977,79,1
2,1j20qq1JyX4,902,0.444,0.054,0.992,0.990,12,0
3,1j20qq1JyX4,902,0.444,0.054,0.992,0.990,17,0
4,1j20qq1JyX4,902,0.444,0.054,0.992,0.990,79,0
...,...,...,...,...,...,...,...,...
2248,1j20qq1JyX4,1796,0.094,0.011,0.967,0.982,79,275
2249,1j20qq1JyX4,1797,0.070,0.025,0.984,0.970,12,275
2250,1j20qq1JyX4,1797,0.070,0.025,0.984,0.970,79,275
2251,1j20qq1JyX4,1798,0.103,0.029,0.983,0.982,11,275


In [44]:
stochastic_df[(stochastic_df["timestamp"] == 902) & (stochastic_df["id"] == "1j20qq1JyX4")]

,id,timestamp,x1,y1,x2,y2,action_id,score
1372911,1j20qq1JyX4,902,0.0362,0.1098,0.5766,0.9783,12,0.9448
1372912,1j20qq1JyX4,902,0.3468,0.0404,0.9629,0.9851,12,0.9393
1372913,1j20qq1JyX4,902,0.4781,0.0229,0.9728,1.0000,80,0.7596
1372914,1j20qq1JyX4,902,0.6034,0.0357,0.9965,0.9724,12,0.7122
1372915,1j20qq1JyX4,902,0.0154,0.1054,0.4940,1.0000,80,0.6344
1372916,1j20qq1JyX4,902,0.0987,0.0539,0.7495,1.0000,80,0.5397
1372917,1j20qq1JyX4,902,0.0265,0.0789,0.5792,1.0000,74,0.4839
1372918,1j20qq1JyX4,902,0.2957,0.0370,0.9676,1.0000,79,0.4685
1372919,1j20qq1JyX4,902,0.5230,0.0249,0.9838,1.0000,17,0.4244
1372920,1j20qq1JyX4,902,0.5675,0.0365,0.9768,1.0000,74,0.3751


In [72]:
len(target_df["action_id"].unique())

80

In [59]:
def match_by_trace_id(target_df, stochastic_df, trace_id):
    dk_activities = []
    sk_activities = []
    sk_scores = []

    target_trace = target_df[target_df["id"] == trace_id]
    stochastic_trace = stochastic_df[stochastic_df["id"] == trace_id]

    for ts in target_trace["timestamp"].unique():
        target_acts = target_trace[target_trace["timestamp"] == ts]
        stochastic_acts = stochastic_trace[stochastic_trace["timestamp"] == ts]

        # If no predictions exist for this timestamp
        if stochastic_acts.empty:
            print(f"cannot find {ts}")
            continue

        target_set = set(target_acts["action_id"])
        stoch_set = set(stochastic_acts["action_id"])
        matching = target_set.intersection(stoch_set)

        # If there is a match
        if matching:
            matched = stochastic_acts[stochastic_acts["action_id"].isin(matching)]
            best_row = matched.loc[matched["score"].idxmax()]

            true_action = int(best_row["action_id"]) - 1   # 0-based
            pred_action = true_action
            pred_score = float(best_row["score"])

            dk_activities.append(true_action)
            sk_activities.append(pred_action)
            sk_scores.append(pred_score)

        # If no match, pick a random GT action
        else:
            # Pick one GT label randomly from target set
            true_action = np.random.choice(list(target_set))
            true_action -= 1  # 0-based

            # Take best prediction
            best_row = stochastic_acts.loc[stochastic_acts["score"].idxmax()]
            pred_action = int(best_row["action_id"]) - 1
            pred_score = float(best_row["score"])

            dk_activities.append(true_action)
            sk_activities.append(pred_action)
            sk_scores.append(pred_score)

    return dk_activities, sk_activities, sk_scores

In [99]:
from tqdm.notebook import tqdm

tokenized_traces = []
sk_traces = []
sk_argmax = []
n_classes = len(activities)
for trace_id in tqdm(target_df["id"].unique()):
    dk_trace, sk_trace, sk_scores = match_by_trace_id(target_df, stochastic_df, trace_id)
    tokenized_traces.append(dk_trace)
    sk_argmax.append(sk_trace)
    sk_traces.append([reconstruct_prob_vector(act, score, n_classes) for act, score in zip(sk_trace, sk_scores)])
    # tokenized_traces.append([x - 1 for x in target_df[target_df["id"] == trace_id]["action_id"].to_numpy()])
    # sk_activities = [x - 1 for x in stochastic_df[stochastic_df["id"] == trace_id]["action_id"].to_numpy()]
    # sk_score = stochastic_df[stochastic_df["id"] == trace_id]["score"].to_numpy()
    # sk_traces.append([reconstruct_prob_vector(act, score, n_classes) for act, score in zip(sk_activities, sk_score)])

  0%|          | 0/64 [00:00<?, ?it/s]

cannot find 1324
cannot find 1543
cannot find 1549
cannot find 1687
cannot find 1417
cannot find 1306
cannot find 1619
cannot find 935
cannot find 936
cannot find 1545
cannot find 1655
cannot find 1636
cannot find 1155
cannot find 928
cannot find 987
cannot find 1271
cannot find 1071
cannot find 1786
cannot find 1251
cannot find 1655
cannot find 1485
cannot find 994
cannot find 1006
cannot find 1047
cannot find 1477


In [100]:
from itertools import chain
from sklearn.preprocessing import OneHotEncoder

n_classes = len(activities)  # 81

dk_one_hot = [np.eye(n_classes, dtype=float)[dk_trace]
              for dk_trace in tokenized_traces]

In [101]:
import pickle as pkl

with open("../data/pickles/ava_unified.pkl", "wb") as f:
    pkl.dump({'target': dk_one_hot, 'stochastic': [np.array(x) for x in sk_traces]}, f)

In [102]:
import pickle as pkl

with open("../data/pickles/ava_unified.pkl", "rb") as f:
    dataset = pkl.load(f)

In [103]:
from src.dataset.dataset import SaladsDataset

salads_dataset = SaladsDataset(dataset['target'], dataset['stochastic'])

In [104]:
import torch

salads_dataset[0][0].shape

torch.Size([888, 81])

In [105]:
salads_dataset[0][1].shape

torch.Size([888, 81])